In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, desc, col
from pyspark.sql.functions import max
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import when

In [3]:
#Build a spark session
spark = SparkSession.builder.appName("Severity_prediction").getOrCreate()

In [4]:
df = spark.read.csv("C:/Users/vmari/OneDrive/Desktop/Lambton/2nd_term/AI/Project_Accidents_severity/US_Accidents_March23.csv", header=True, inferSchema=False)



In [5]:
df.show()

+----+-------+--------+-------------------+-------------------+------------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  ID| Source|Severity|         Start_Time|           End_Time|         Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|A

In [13]:
Description=df.select("Description")
Description.show(200,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+
|Description                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------+
|Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4.                                                  |
|Accident on Brice Rd at Tussing Rd. Expect delays.                                                                                     |
|Accident on OH-32 State Route 32 Westbound at Dela Palma Rd. Expect delays.                                                            |
|Accident on I-75 Southbound at Exits 52 52B US-35. Expect delays.                                                                      |
|Accident on McEwen Rd at OH-725 M

Precipitation(in)= precipitation acumulation amount

In [7]:
Precipitation_exploaration=df.select("Precipitation(in)","Humidity(%)","Severity")
Precipitation_exploaration.show()

+-----------------+-----------+--------+
|Precipitation(in)|Humidity(%)|Severity|
+-----------------+-----------+--------+
|             0.02|       91.0|       3|
|              0.0|      100.0|       2|
|             NULL|      100.0|       2|
|             NULL|       96.0|       3|
|             NULL|       89.0|       2|
|             0.03|       97.0|       3|
|             NULL|      100.0|       2|
|             NULL|      100.0|       3|
|             NULL|       99.0|       2|
|             0.02|      100.0|       3|
|             NULL|       93.0|       3|
|             0.02|      100.0|       3|
|             NULL|      100.0|       2|
|             NULL|       89.0|       2|
|             0.02|      100.0|       2|
|             NULL|      100.0|       2|
|             NULL|       99.0|       2|
|             NULL|       89.0|       2|
|             NULL|       93.0|       2|
|             NULL|       89.0|       2|
+-----------------+-----------+--------+
only showing top

In [8]:
max_value_p = Precipitation_exploaration.select(max("Precipitation(in)")).collect()[0][0]
print("Max value of precipitation(in) =",max_value_p)



Max value of precipitation(in) = 9.99


In [9]:
value_counts_p = Precipitation_exploaration.groupBy("Precipitation(in)").count().orderBy("Precipitation(in)")

value_counts_p.show(200)

+-----------------+-------+
|Precipitation(in)|  count|
+-----------------+-------+
|             NULL|2203586|
|              0.0|4991718|
|             0.01| 151010|
|             0.02|  74008|
|             0.03|  50055|
|             0.04|  37300|
|             0.05|  29167|
|             0.06|  23958|
|             0.07|  19353|
|             0.08|  16138|
|             0.09|  14307|
|              0.1|  11916|
|             0.11|  10166|
|             0.12|   8805|
|             0.13|   7847|
|             0.14|   6775|
|             0.15|   5841|
|             0.16|   5169|
|             0.17|   4746|
|             0.18|   4222|
|             0.19|   3702|
|              0.2|   3348|
|             0.21|   2980|
|             0.22|   2654|
|             0.23|   2581|
|             0.24|   2510|
|             0.25|   1985|
|             0.26|   1949|
|             0.27|   1731|
|             0.28|   1693|
|             0.29|   1305|
|              0.3|   1355|
|             0.31| 

Wind Chill (F)= measure of how cold the air feels to the human body under windy conditions

In [10]:
Wind_Chill_exploaration=df.select("Wind_Chill(F)","Temperature(F)","Severity")
Wind_Chill_exploaration.show()

+-------------+--------------+--------+
|Wind_Chill(F)|Temperature(F)|Severity|
+-------------+--------------+--------+
|         NULL|          36.9|       3|
|         NULL|          37.9|       2|
|         33.3|          36.0|       2|
|         31.0|          35.1|       3|
|         33.3|          36.0|       2|
|         35.5|          37.9|       3|
|         31.0|          34.0|       2|
|         31.0|          34.0|       3|
|         NULL|          33.3|       2|
|         33.8|          37.4|       3|
|         30.7|          35.6|       3|
|         33.8|          37.4|       3|
|         NULL|          33.8|       2|
|         31.1|          36.0|       2|
|         33.8|          37.4|       2|
|         NULL|          33.8|       2|
|         NULL|          35.6|       2|
|         31.1|          36.0|       2|
|         32.1|          37.4|       2|
|         30.3|          36.0|       2|
+-------------+--------------+--------+
only showing top 20 rows



In [11]:
value_counts_wc = Wind_Chill_exploaration.groupBy("Wind_Chill(F)").count().orderBy("Wind_Chill(F)")

value_counts_wc.show(200)

+-------------+-------+
|Wind_Chill(F)|  count|
+-------------+-------+
|         NULL|1999019|
|         -0.1|    111|
|         -0.2|    399|
|         -0.3|     60|
|         -0.4|    294|
|         -0.5|    127|
|         -0.6|    130|
|         -0.7|     26|
|         -0.8|    246|
|         -0.9|    509|
|         -1.0|   2883|
|         -1.1|    128|
|         -1.3|    274|
|         -1.4|    153|
|         -1.5|     79|
|         -1.6|    135|
|         -1.7|    107|
|         -1.8|    183|
|         -1.9|    152|
|        -10.0|   1660|
|        -10.1|     28|
|        -10.2|     86|
|        -10.3|     77|
|        -10.4|    147|
|        -10.5|     88|
|        -10.6|     66|
|        -10.7|     44|
|        -10.8|     99|
|        -10.9|     47|
|        -11.0|   1450|
|        -11.1|     11|
|        -11.2|     52|
|        -11.3|    128|
|        -11.4|     27|
|        -11.5|     62|
|        -11.6|    158|
|        -11.7|     88|
|        -11.8|     76|
|        -11.9| 